In [ ]:
import urllib
import pandas as pd
from pandas import read_csv
import scipy as sp
import numpy as np
from numpy import append, linspace, reshape, mean ,std, sum, array, dot, meshgrid, rint
from pylab import plot, scatter, xlabel, ylabel, contour,figure, show, axes, legend, title
from scipy.optimize import fmin_bfgs,fmin_cg,fmin_ncg,minimize
import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
import  scipy.stats as stats

import warnings
warnings.filterwarnings('ignore')

In [ ]:
dataset = np.loadtxt('heart_statlog.txt',delimiter=",")

dfs = pd.DataFrame(dataset)
ds = dfs.loc[np.random.choice(dfs.index, 270)]
ds = ds.as_matrix()
X = ds[:,0:12]
y = ds[:,13]
y[y==1] = 0
y[y==2] = 1
dataset

In [ ]:
def sigmoid(z,cg,cf):
    g = np.zeros(z.shape)
    g = cg+(1-cg-cf)/(1+np.exp(-z))
    return g

def sigmoid0(z):
    g = np.zeros(z.shape)
    g = 1/(1+np.exp(-z))
    return g

def sig_diff(z,cg,cf):
    g = np.zeros(z.shape)
    s0 = sigmoid0(z)
    s = sigmoid(z,cg,cf)
    g0 = s0*(1-s0)
    g = s*(1-s)
    return dot(g0.T,1/g)

def poly(X,p):
    X_poly = X
    if p >= 2:
        for k in xrange(1,p):
            X_poly = np.column_stack((X_poly, np.power(X,k+1)))
    return X_poly

def costFunction(theta, X, y, cg,cf):
    #theta = np.array(theta)
    #theta = reshape(theta,(theta.size,1))
    pred = sigmoid(dot(X,theta),cg,cf)
    J = mean(-y*log(pred)-(1-y)*log(1-pred))
    return J

def gradientC(theta, X, y, cg,cf):
    m = y.size

    pred = sigmoid(dot(X,theta),cg,cf)
    subgrad = (1-cg-cf)
    grad = dot(X.T,pred-y)/m
    postgrad = grad*subgrad
    return postgrad.flatten()

def predict(theta, X, cg,cf, threshold=0.5):
    p = sigmoid(X.dot(theta.T),cg,cf) >= threshold
    return p

In [ ]:
def trainLRegC(X, y, cg,cf):
    theta, theta_i = fmin_cg(costFunction,initial_theta,gtol=1e-5,fprime = gradientC,args=(X,y,cg,cf),retall = 1, disp = 0, maxiter=500)
    return theta

def trainLReg(X, y, cg,cf):
    theta, theta_i = fmin_cg(costFunction,initial_theta,gtol=1e-5,fprime = gradient,args=(X,y,0,0),retall = 1, disp = 0, maxiter=500)
    return theta

def validationCurveGFC(X, y, Xval, yval, c_vecg, c_vecf):
    error_train = np.zeros((len(c_vecg), len(c_vecf)))
    error_val = np.zeros((len(c_vecg), len(c_vecf)))
    for i in range(len(c_vecg)):
        for j in range(len(c_vecf)):
            cg = c_vecg[i]
            cf = c_vecf[j]
            theta = trainLRegC(X, y, cg, cf)
            error_train[i,j] = costFunction(theta, X,    y,   0,0)
            error_val[i,j]   = costFunction(theta, Xval, yval, 0,0)         
    return error_train, error_val



In [ ]:
random_state = 33

K = 80
ngf = 20
step = 0.005

c_vecg = np.linspace(0.0,0.15,ngf+1)
c_vecf = np.linspace(0.0,0.15,ngf+1)

error_trainGFKC = np.zeros((len(c_vecg), len(c_vecf),K))
error_valGFKC = np.zeros((len(c_vecg), len(c_vecf),K))
error_trainGFMC = np.zeros((len(c_vecg), len(c_vecf),1))
error_valGFMC = np.zeros((len(c_vecg), len(c_vecf),1))

In [ ]:
M = 100
minM = 20

minTC = np.zeros((M, 1))
minVC = np.zeros((M, 1))
minT0 = np.zeros((M, 1))
minV0 = np.zeros((M, 1))
CfTC = np.zeros((M, 1))
CgTC = np.zeros((M, 1))
CfVC = np.zeros((M, 1))
CgVC = np.zeros((M, 1))
ftC = np.zeros((M, 1))
gtC = np.zeros((M, 1))
fvC = np.zeros((M, 1))
gvC = np.zeros((M, 1))

for m in range(minM, M + 1):
    for k in range(K):
        dataset = dataset[~np.isnan(dataset).any(axis=1)]
       
        dfs = pd.DataFrame(dataset)
        ds = dfs.loc[np.random.choice(dfs.index, m)]
        ds = ds.as_matrix()
        X = ds[:,0:12]
        y = ds[:,13]
        y[y==1] = 0
        y[y==2] = 1
    
    
        X = StandardScaler().fit_transform(X)
        X = append(ones((X.shape[0],1)),X,axis=1)
    
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.2, random_state=random_state)
                    

        initial_theta = np.zeros((X_train.shape[1]))
  
        error_trainGFC, error_valGFC = validationCurveGFC(X_train, y_train, X_val, y_val, c_vecg, c_vecf)
        error_trainGFKC = np.dstack((error_trainGFKC,error_trainGFC ))
        error_valGFKC = np.dstack((error_valGFKC,error_valGFC )) 

        
        print (k)

    
    error_trainGFKC[error_trainGFKC == inf] = np.nan
    error_trainGFKC[error_trainGFKC == -inf] = np.nan
    col_mean1 = np.nanmean(error_trainGFKC,axis=2)
    inds = np.where(np.isnan(error_trainGFKC))
    error_trainGFKC[inds]=np.take(col_mean1,inds[0])
    error_trainGFMC = np.nanmean(error_trainGFKC, axis=2)


    error_valGFKC[error_valGFKC == inf] = np.nan
    error_valGFKC[error_valGFKC == -inf] = np.nan
    col_mean2 = np.nanmean(error_valGFKC,axis=2)
    inds = np.where(np.isnan(error_valGFKC))
    error_valGFKC[inds]=np.take(col_mean2,inds[0])
    error_valGFMC = np.nanmean(error_valGFKC, axis=2)
    
    #***
    error_valGFMC = np.nanmean(error_valGFKC, axis=2)
    sdV = np.nanstd(error_valGFKC, axis=2)
    print(sdV)
    
    
    minTC[m-minM] = numpy.amin(error_trainGFMC[np.nonzero(error_trainGFMC)])
    minT0[m-minM] = error_trainGFMC[0,0]
    ites = np.where(error_trainGFMC == minTC[m-minM])
    CgTC[m-minM] = ites[0][0]
    CfTC[m-minM] = ites[1][0]
    
    ftC[m-minM] = c_vecf[int(CfTC[m-minM])]
    gtC[m-minM] = c_vecg[int(CgTC[m-minM])]

    minVC[m-minM] = numpy.amin(error_valGFMC[np.nonzero(error_valGFMC)])
    minV0[m-minM] = error_valGFMC[0,0]
    ites = np.where(error_valGFMC == minVC[m-minM])
    CgVC[m-minM] = ites[0][0]
    CfVC[m-minM] = ites[1][0]
    
    #***
    sdV0[m-minM] = sdV[0,0]
    sdVC[m-minM] = sdV[int(CfVC[m-minM]),int(CgVC[m-minM])]

    fvC[m-minM] = c_vecf[int(CfVC[m-minM])]
    gvC[m-minM] = c_vecg[int(CgVC[m-minM])]
    
    print(m)
    

In [ ]:
cols = ['gtC','ftC','gvC','fvC','minT0','minV0','minTC','minVC','sdV0','sdVC']
df = pd.DataFrame({"gtC" : gtC[0:(M-minM),0], "ftC" : ftC[0:(M-minM),0],"gvC" : gvC[0:(M-minM),0], "fvC" : fvC[0:(M-minM),0],
                  "minT0" : minT0[0:(M-minM),0], "minV0" : minV0[0:(M-minM),0],"minTC" : minTC[0:(M-minM),0], "minVC" : minVC[0:(M-minM),0],
                  "sdV0" : sdV0[0:(M-minM),0],"sdVC" : sdVC[0:(M-minM),0]})
df.to_csv("grad-curve-batch-heart_sd.csv",index=False)